<a href="https://colab.research.google.com/github/moustafa123/teacher-assistant-chatbot/blob/main/TeacherAssistantBloomTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install datasets

In [ ]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

In [ ]:
# Get the tokenizer and model from bloomz-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m").to("cuda")

In [ ]:
# Load json prompt dataset
dataset = load_dataset("json", data_files="prompts.json")

In [ ]:
# Print dataset
dataset

In [ ]:
# Function to prepare the training data
def prepare_train_data(data):
    # prompt + completion
    #text_input = data['prompt'] + ' ' + data['completion']
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding=True)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

In [ ]:
# Create train entries in dataset
train_dataset = dataset['train'].map(prepare_train_data, batched=True, remove_columns=["text"])

In [ ]:
# Print train dataset
train_dataset

In [ ]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

In [ ]:
# Create the Training Arguments
training_arguments = TrainingArguments(
    'LegalQA-bloom-560m-dev',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [ ]:
# Create the trainer based on the Training Arguments
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

In [ ]:
# Train the model on the dataset
trainer.train()

In [ ]:
# Save a local copy of the model to be used in the future
trainer.save_model()